In [9]:
# 크롬 드라이버
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import json

driver = webdriver.Chrome()
driver.get("https://www.data.go.kr/index.do")

time.sleep(3)

# 원격으로 검색하기
searchValue = input("어떤 공공데이터를 찾으시나요? ")
searchPage = input("수집하실 페이지 범위를 입력하세요(예: 1~4): ")
searchPage = list(map(int, searchPage.split("~")))

searchInput = driver.find_element(by=By.CSS_SELECTOR, value="#keyword")
searchInput.send_keys(searchValue)              # input을 통해 입력한 값이 검색창에 입력됨
searchInput.send_keys(Keys.RETURN)              # 자동으로 enter가 되어 검색된 화면으로 이동

time.sleep(2)

driver.find_element(by=By.XPATH, value='//*[@id="dTypeAPI"]/a').click()  # 오픈 API 페이지로 이동버튼 클릭
time.sleep(1)

data_list = []
for i in range(searchPage[0], searchPage[1] + 1):
    if(i > 1):      # 1페이지는 페이지 버튼 클릭 제외
        driver.find_element(by=By.XPATH, value=f'//*[@id="apiDataList"]/nav/a[{i+1}]').click()  # 페이지 이동버튼 클릭
        time.sleep(3)

    divs = driver.find_elements(by=By.CSS_SELECTOR, value=".result-list ul li")

    for div in divs:
        title = div.find_element(by=By.CSS_SELECTOR, value=".title").get_attribute(name="innerText")
        provider_organization = div.find_element(by=By.CSS_SELECTOR, value=".data").get_attribute(name="innerText")
        update_date = div.find_element(by=By.CSS_SELECTOR, value=".info-data p:nth-child(2) .data").get_attribute(name="innerText")
        hits = div.find_element(by=By.CSS_SELECTOR, value=".info-data p:nth-child(3) .data").get_attribute(name="innerText")
        request_count = div.find_element(by=By.CSS_SELECTOR, value=".info-data p:nth-child(4) .data").get_attribute(name="innerText")
        keyword = div.find_element(by=By.CSS_SELECTOR, value=".info-data p:nth-child(5)").text.split("키워드 ")[1]

        data = {
            "title": title,
            "provider_organization": provider_organization,
            "update_date": update_date,
            "hits": hits,
            "request_count": request_count,
            "keyword": keyword
        }
        data_list.append(data)

with open("data_list.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, indent=4, ensure_ascii=False)